In [ ]:
import pandas as pd

In [ ]:
df_street_rating = pd.read_csv('data/Street_Pavement_Rating_20240310.csv', low_memory=False)
df_protected = pd.read_csv('data/NYCDOT_ProtectedStreets_332024.csv')

# Merging with Street Rating Dataset

In [ ]:
# Replace with your methodology for loading the data

In [ ]:
df['geometry'] = gpd.points_from_xy(df['LONGITUDE'], df['LATITUDE'])
gdf = gpd.GeoDataFrame(df,geometry=df['geometry'],crs='EPSG:4326')
gdf.head()

In [ ]:
gdf_street = gpd.read_file('data/Street_Pavement_Rating_20240310.csv')
gdf_street.crs = 'EPSG:4326'
gdf_street['the_geom'] = gdf_street['the_geom'].apply(wkt.loads)
gdf_street = gdf_street.set_geometry('the_geom')

In [ ]:
gdf_merge = gpd.sjoin_nearest(gdf, gdf_street, how='left', distance_col='distance')

# Merging with Protected Streets Dataset

In [ ]:
def check_protected_status(row, df_protected):
    protected_rows = df_protected[df_protected['protected'] == row['protected']]
    if not protected_rows.empty:
        protected_row = protected_rows.iloc[0]
        crash_date = pd.to_datetime(row['CRASH DATE'])
        completed_on_date = pd.to_datetime(protected_row['Completed on Date'])
        protected_until_date = pd.to_datetime(protected_row['Protected Until Date'])
        if crash_date > completed_on_date and crash_date < protected_until_date:
            return True, completed_on_date  # Return True and completed on date if protected
        else:
            return False, None  # Return False if not protected
    else:
        return False, None  # Return False if protected data is not available

In [ ]:
# Remove leading and trailing whitespace from street names
df.loc[:, 'ON STREET NAME'] = df['ON STREET NAME'].str.strip()
df.loc[:, 'CROSS STREET NAME'] = df['CROSS STREET NAME'].str.strip()

# Concatenate two columns to create pairs in one order
df['INTERSECTION'] = df[['ON STREET NAME', 'CROSS STREET NAME']].astype(str).apply(sorted, axis=1).apply(' and '.join)

df_protected['protected'] = df_protected[[' On Street', 'From Street']].astype(str).apply(sorted, axis=1).apply(' and '.join)

# Mergedf with df_protected based on the protected column
df =df[df['INTERSECTION'] != 'nan and nan']

merged_df = pd.merge(df, df_protected, on='protected', suffixes=('_crashes', '_protected'))

# Convert date columns to datetime if they are not already
merged_df['CRASH DATE'] = pd.to_datetime(merged_df['CRASH DATE'])
merged_df['Completed on Date'] = pd.to_datetime(merged_df['Completed on Date'])
merged_df['Protected Until Date'] = pd.to_datetime(merged_df['Protected Until Date'])


# Check if crash occurred during protected status
merged_df['PROTECTED'] = (merged_df['CRASH DATE'] > merged_df['Completed on Date']) # & \
                        #  (merged_df['CRASH DATE'] < merged_df['Protected Until Date'])

# Fill False for rows where protected not found
#merged_df['PROTECTED'].fillna(False, inplace=True)

# Extract PROTECTED column todf DataFrame
df['PROTECTED'] = merged_df['PROTECTED']
df['PROTECTED SINCE'] = merged_df['Completed on Date']
df.head()